In [1]:
from datetime import datetime
import requests
import numpy as np
import pandas as pd
import yaml
import re
from bs4 import BeautifulSoup
import wikitextparser as wtp
from ratelimit import rate_limited
from operator import *

def save_obj(obj, name ):
    import pickle
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name ):
    import pickle
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [2]:
def parseSquad(tourdict):
    '''
    input: title of tournament
    output: dictionary of participants of that tournament
    '''
    json = tourdict['wikitext']
    wiki = json.get('parse').get('wikitext').get('*')
    wiki = re.sub('\n', '', wiki)
    parsed = wtp.parse(wiki)
    templates = parsed.templates
    squad = {}
    for t in templates:
        if str(t)[:11] == '{{TeamCard|':
            for ar in t.arguments:
                if (str(ar)[:5] == '|team'):
                    teamname = re.sub(r'\W+', '', ar.value).lower()
                    squad[teamname] = {}
                for i in range(1, 6):
                    if (str(ar)[:4] == '|p' + str(i) + '='): 
                        pid = re.sub(r'\W+', '', ar.value).lower()
                        try:
                            pid = AlternateIdDict[re.sub(r'\W+', '', ar.value).lower()]
                        except KeyError:
                            pid = re.sub(r'\W+', '', ar.value).lower()
                        squad[teamname]['pos' + str(i)] = re.sub(r'\W+', '', pid).lower()
                    if (str(ar)[:7] == '|p' + str(i) + 'link'):
                        squad[teamname]['pos' + str(i)] = squad[teamname]['pos' + str(i)] + ' haslink'
    return squad

In [3]:
def getEloDict(tourdict):
    return (tourdict['startelodict'], tourdict['endelodict'])

In [4]:
def compareAndPut(df, squad, ifd, startelodict, endelodict):
    for lqn, pdc in squad.items():
        try:
            teamName = comparedict[lqn]
        except KeyError:
            teamName = lqn
        j = 0
        for srt in startelodict:
            for ert in endelodict:
                if (teamName == re.sub(r'\W+', '', srt['teamName']).lower()) & (teamName == re.sub(r'\W+', '', ert['teamName']).lower()):
                    j += 1
                    for pos, id in pdc.items():
                        df = df.append({
                            'team': teamName,
                            'pos': pos,
                            'id': id,
                            'tour': ifd['title'],
                            'prizeusd': ifd['prize'],
                            'start': ifd['start'],
                            'end': ifd['end'],
                            'startelo': srt['elo64']['current'],
                            'startglicko2mu': srt['glicko2']['mu'],
                            'startglicko2phi': srt['glicko2']['phi'],
                            'startglicko2rating': srt['glicko2']['rating'],
                            'endelo': ert['elo64']['current'],
                            'endglicko2mu': ert['glicko2']['mu'],
                            'endglicko2phi': ert['glicko2']['phi'],
                            'endglicko2rating': ert['glicko2']['rating']
                        }, ignore_index=True)
        if j == 0:
            for pos, id in pdc.items():
                if id == '':
                    continue
                else:
                    df = df.append({
                        'team': teamName,
                        'pos': pos,
                        'id': id,
                        'tour': ifd['title'],
                        'prizeusd': ifd['prize'],
                        'start': ifd['start'],
                        'end': ifd['end'],
                        'startelo': np.nan,
                        'startglicko2mu': np.nan,
                        'startglicko2phi': np.nan,
                        'startglicko2rating': np.nan,
                        'endelo': np.nan,
                        'endglicko2mu': np.nan,
                        'endglicko2phi': np.nan,
                        'endglicko2rating': np.nan
                    }, ignore_index=True)
        
    return df

In [5]:
alltour = load_obj('/dict/touranddate')
comparedict = load_obj('/dict/lqtodat')
AlternateIdDict = load_obj('/dict/AlternateIdDict')

In [6]:
endti2 = '2012-09-02'
endti7 = '2017-08-12'

In [7]:
%%time
errortitle = []
df = pd.DataFrame(columns=['team', 'pos', 'id', 'tour', 'prizeusd', 'start', 'end',
                           'startelo', 'startglicko2mu', 'startglicko2phi', 'startglicko2rating', 'endelo', 'endglicko2mu', 'endglicko2phi', 'endglicko2rating'])
for turl, ifd in alltour.items():
    try:
        if (pd.to_datetime(ifd['start'], format='%d/%m/%Y') > pd.to_datetime(endti2)) & (pd.to_datetime(ifd['end'], format='%d/%m/%Y') <= pd.to_datetime(endti7)):
            tourdict = load_obj('forelo/' + re.sub(r'\W+', '', ifd['title']).lower() )
            ptc = parseSquad(tourdict)
            startelodict, endelodict = getEloDict(tourdict)
            df = compareAndPut(df, ptc, ifd, startelodict, endelodict)
    except:
        print(turl)
        errortitle = errortitle.append(turl)

Wall time: 1min 38s


In [8]:
#Clean Data
df = df[df['id'] != '']
df.loc[(df['id'] == 'g haslink'), 'id'] = 'g'
df.loc[(df['id'] == 'noone haslink'), 'id'] = 'noone'
df.loc[(df['id'].str[:8] == 'paparazi'), 'id'] = 'paparazi'
df.pos = pd.to_numeric(df.pos.str[-1])

In [9]:
df[['team', 'pos', 'id', 'tour', 'prizeusd', 'start', 'startelo', 'end', 'endelo']].to_csv('elo.csv', encoding='utf-8', index=False)
#df[['team', 'pos', 'id', 'tour', 'prizeusd', 'start', 'startglicko2mu', 'startglicko2phi', 'startglicko2rating', 'end', 'endglicko2mu', 'endglicko2phi', 'endglicko2rating']].to_csv('glicko.csv', encoding='utf-8', index=False)